# This is program computes specific yield

In [11]:
import os
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
from shapely.geometry import mapping

temp=pd.date_range(start="2004-01", end="2023-01", freq='M').strftime('%Y-%m')
total_ts=pd.to_datetime(temp)

# Load the India boundary shapefile
US = gpd.read_file(r'/home/bramha/Desktop/5hk/Codes/Specific_yield/example_data_for_sy/1_US_outline/1_US_Outline.shp')

# Directory where the shapefiles are stored
shapefile_directory = '/home/bramha/Desktop/5hk/Codes/Specific_yield/example_data_for_sy/2_mascon_shp/'

# Get all shapefiles in the directory (assuming they are all .shp files)
shapefiles = glob.glob(os.path.join(shapefile_directory, "*.shp"))

#GRACE JPL data
GRACE_JPL_file = glob.glob('/home/bramha/Desktop/5hk/Codes/Specific_yield/example_data_for_sy/3_GRACE_JPL/GRCTellus.JPL.200204_202407.GLO.RL06.3M.MSCNv04CRI.nc')


# Directory where SM and GWL data are stored
sm_directory = '/home/bramha/Desktop/5hk/Codes/Specific_yield/example_data_for_sy/4_soil_moisture/'
gwl_directory = '/home/bramha/Desktop/5hk/Codes/Specific_yield/example_data_for_sy/5_groundwater_level/'

# Prepare lists to save results
mean_sy_results = []
mean_sy_uncer_results = []

# Loop through each shapefile
for shapefile in shapefiles:
    shapefile_name = os.path.basename(shapefile).replace('.shp', '')
    
    # Extract the mascon number from the shapefile name (assumes format 'mascon<NN>')
    mascon_number = shapefile_name.split('_')[-1]  # Extract last part of the filename (e.g., 'mascon11')
    
    # Extract the numerical part (i.e., remove the 'mascon' prefix)
    mascon_number = mascon_number.replace('mascon', '').zfill(2)  # Ensure it's always 2 digits (e.g., '11', '12', etc.)

    # Load and clip mascon shapefile (MS)
    MS = gpd.read_file(shapefile)  # Load mascon shapefile
    
    # Select SM and GWL files that contain the mascon_number
    sm_files = glob.glob(f"{sm_directory}/*{mascon_number}*.csv")
    gwl_files = glob.glob(f"{gwl_directory}/*{mascon_number}*.csv")

    if not sm_files:
        print(f"No SM file found for mascon {mascon_number}")
        continue
    
    if not gwl_files:
        print(f"No GWL file found for mascon {mascon_number}")
        continue

    # Load SM and GWL data
    SM = pd.read_csv(sm_files[0])  # Assuming only one matching file
    GWL = pd.read_csv(gwl_files[0])  # Assuming only one matching file

    # GRACE
    mascon_stack = xr.open_mfdataset(GRACE_JPL_file)
    mascon_stack = mascon_stack.sel(time=slice('2004-01-01', '2022-12-31'))
    mascon_stack.coords['lon'] = (mascon_stack.coords['lon'] + 180) % 360 - 180
    mascon_stack = mascon_stack.sortby(mascon_stack.lon)
    mascon_stack['lwe_thickness'] = mascon_stack['lwe_thickness'] * 0.01
    mascon_stack['uncertainty'] = mascon_stack['uncertainty'] * 0.01
    ds = mascon_stack.resample(time="M").mean(dim="time")
    bands = ['lwe_thickness', 'uncertainty']
    ds_bands = ds[bands]
    ds_bands.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    ds_bands.rio.write_crs("epsg:4326", inplace=True)
    ds_India = ds_bands.rio.clip(US.geometry.apply(mapping), US.crs, all_touched=True)
    ds_select = ds_India.rio.clip(MS.geometry.apply(mapping), MS.crs, all_touched=False) 

    GRACE_TWSA_avg_monthly = ds_select.mean(['lon', 'lat'])

    # Perform GRACE-GWSA and GWL operations (same logic as original code)
    gr_gw_sm_ts_old = pd.DataFrame({
        'Time': total_ts.values,
        'gwl(m)': GWL['gwl_median(m)'].values,
        'GRACE_TWSA(m)': GRACE_TWSA_avg_monthly.lwe_thickness.values,
        'SMA_EWH(m)': SM['sma_EWH(m)'].values
    })
    gr_gw_sm_ts_old['GRACE_GWSA(m)']=gr_gw_sm_ts_old['GRACE_TWSA(m)']-gr_gw_sm_ts_old['SMA_EWH(m)']
    gr_gw_sm_ts_old['GWL_Uncer(m)']=GWL['gwl_median_abs_error(m)'].values #gwl_uncer_array
    gr_gw_sm_ts_old['GRACE_TWSA_Uncer(m)']=GRACE_TWSA_avg_monthly.uncertainty.values
    gr_gw_sm_ts_old['sma_EWH_Uncer(m)']=SM['sma_EWH_uncer(m)'].values
    gr_gw_sm_ts_old['GRACE_GWSA_Uncer(m)']=np.sqrt(gr_gw_sm_ts_old['GRACE_TWSA_Uncer(m)']**2+gr_gw_sm_ts_old['sma_EWH_Uncer(m)']**2)
    
    gr_gw_sm_ts = gr_gw_sm_ts_old.dropna()
    gr_gw_sm_ts = gr_gw_sm_ts.reset_index()
    
    if gr_gw_sm_ts.empty:
        print(f"Skipping mascon {mascon_number} due to missing or invalid data.")
        continue

    # Extract year and month
    gr_gw_sm_ts['Year'] = gr_gw_sm_ts['Time'].dt.year
    gr_gw_sm_ts['Month'] = gr_gw_sm_ts['Time'].dt.month  # Add the month column

    # --- Handling Maximum GRACE_GWSA(m) ---
    # Get max GRACE_GWSA values per year
    max_Grace_gwa = gr_gw_sm_ts.groupby(['Year'])["GRACE_GWSA(m)"].max().reset_index()

    # Handle cases where the max value occurs in more than one month for a given year
    max_Grace_gwa_result = []
    for _, row in max_Grace_gwa.iterrows():
        max_value = row['GRACE_GWSA(m)']
        year = row['Year']

        # Find all rows in the same year with this maximum value
        matching_rows = gr_gw_sm_ts[(gr_gw_sm_ts['Year'] == year) & (gr_gw_sm_ts['GRACE_GWSA(m)'] == max_value)]

        if not matching_rows.empty:
            # If there are multiple months with the same value, select the latest month
            latest_row = matching_rows[matching_rows['Month'] == matching_rows['Month'].max()]
            max_Grace_gwa_result.append(latest_row)

    if max_Grace_gwa_result:
        result_max_gwa = pd.concat(max_Grace_gwa_result)
        max_GWSA_uncer = result_max_gwa['GRACE_TWSA_Uncer(m)'].values
    else:
        print(f"No valid max GRACE_GWSA data for mascon {mascon_number}")
        continue

    # --- Handling Maximum GWL(m) ---
    max_gwl = gr_gw_sm_ts.groupby(['Year'])["gwl(m)"].max().reset_index()
    max_gwl_result = []
    for _, row in max_gwl.iterrows():
        max_value = row['gwl(m)']
        year = row['Year']

        matching_rows = gr_gw_sm_ts[(gr_gw_sm_ts['Year'] == year) & (gr_gw_sm_ts['gwl(m)'] == max_value)]
        
        if not matching_rows.empty:
            latest_row = matching_rows[matching_rows['Month'] == matching_rows['Month'].max()]
            max_gwl_result.append(latest_row)

    if max_gwl_result:
        result_max_gwl = pd.concat(max_gwl_result)
        max_gwl_uncer = result_max_gwl['GWL_Uncer(m)'].values
    else:
        print(f"No valid max GWL data for mascon {mascon_number}")
        continue

    # --- Handling Minimum GRACE_GWSA(m) ---
    min_Grace_gwa = gr_gw_sm_ts.groupby(['Year'])["GRACE_GWSA(m)"].min().reset_index()
    min_Grace_gwa_result = []
    for _, row in min_Grace_gwa.iterrows():
        min_value = row['GRACE_GWSA(m)']
        year = row['Year']

        matching_rows = gr_gw_sm_ts[(gr_gw_sm_ts['Year'] == year) & (gr_gw_sm_ts['GRACE_GWSA(m)'] == min_value)]
        
        if not matching_rows.empty:
            latest_row = matching_rows[matching_rows['Month'] == matching_rows['Month'].max()]
            min_Grace_gwa_result.append(latest_row)

    if min_Grace_gwa_result:
        result_min_gwa = pd.concat(min_Grace_gwa_result)
        min_GWSA_uncer = result_min_gwa['GRACE_TWSA_Uncer(m)'].values
    else:
        print(f"No valid min GRACE_GWSA data for mascon {mascon_number}")
        continue

    # --- Handling Minimum GWL(m) ---
    min_gwl = gr_gw_sm_ts.groupby(['Year'])["gwl(m)"].min().reset_index()
    min_gwl_result = []
    for _, row in min_gwl.iterrows():
        min_value = row['gwl(m)']
        year = row['Year']

        matching_rows = gr_gw_sm_ts[(gr_gw_sm_ts['Year'] == year) & (gr_gw_sm_ts['gwl(m)'] == min_value)]
        
        if not matching_rows.empty:
            latest_row = matching_rows[matching_rows['Month'] == matching_rows['Month'].max()]
            min_gwl_result.append(latest_row)

    if min_gwl_result:
        result_min_gwl = pd.concat(min_gwl_result)
        min_gwl_uncer = result_min_gwl['GWL_Uncer(m)'].values
    else:
        print(f"No valid min GWL data for mascon {mascon_number}")
        continue
    ts_sy_uncer=pd.DataFrame({'Year':max_Grace_gwa['Year'].values,'gwl_min(m)':min_gwl['gwl(m)'].values,'gwl_max(m)':max_gwl['gwl(m)'].values,'gwsa_min(m)':min_Grace_gwa['GRACE_GWSA(m)'].values,'gwsa_max(m)':max_Grace_gwa['GRACE_GWSA(m)'].values,'gwl_min_uncer(m)':min_gwl_uncer,'gwl_max_uncer(m)':min_gwl_uncer,'gwsa_min_uncer(m)':min_GWSA_uncer,'gwsa_max_uncer(m)':max_GWSA_uncer})

    # Calculate recharge, gwlc, annual_sy, etc.
    ts_sy_uncer['recharge(m)'] = ts_sy_uncer['gwsa_max(m)'] - ts_sy_uncer['gwsa_min(m)']
    ts_sy_uncer['gwlc(m)'] = ts_sy_uncer['gwl_max(m)'] - ts_sy_uncer['gwl_min(m)']
    ts_sy_uncer['annual_sy'] = ts_sy_uncer['recharge(m)'] / ts_sy_uncer['gwlc(m)']
    ts_sy_uncer['recharge_uncer(m)'] = np.sqrt(ts_sy_uncer['gwsa_max_uncer(m)']**2 + ts_sy_uncer['gwsa_min_uncer(m)']**2)
    ts_sy_uncer['gwlc_uncer(m)'] = np.sqrt(ts_sy_uncer['gwl_max_uncer(m)']**2 + ts_sy_uncer['gwl_min_uncer(m)']**2)
    ts_sy_uncer['annual_sy_uncer(m)'] = ts_sy_uncer['annual_sy'] * np.sqrt(
        (ts_sy_uncer['recharge_uncer(m)'] / ts_sy_uncer['recharge(m)'])**2 +
        (ts_sy_uncer['gwlc_uncer(m)'] / ts_sy_uncer['gwlc(m)'])**2
    )

    # Apply IQR for filtering
    Q1 = ts_sy_uncer['annual_sy'].quantile(0.25)
    Q3 = ts_sy_uncer['annual_sy'].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    filtered_df = ts_sy_uncer[(ts_sy_uncer['annual_sy'] >= lower_bound) & (ts_sy_uncer['annual_sy'] <= upper_bound)]

    # Save filtered result
    filtered_df['Mascon_Number'] = mascon_number
    filtered_df.to_csv(f'/home/bramha/Desktop/5hk/Codes/Specific_yield/example_data_for_sy/output/{shapefile_name}_after_IQR.csv', index=False)